In [1]:
import os
import subprocess
import io
import pandas as pd
import csv
import asyncio
from tqdm import tqdm
import concurrent.futures as mp


In [6]:
# Directory containing the pcapng files
pcapng_dir = "C:\\Users\Philipp\OneDrive - UT Cloud\Data_literacy_mensa\Mensa_10-01-24\pcapng"

# Directory to save the csv files
csv_dir = "C:\\Users\Philipp\OneDrive - UT Cloud\Data_literacy_mensa\Mensa_10-01-24\csv"

# File name for merged CSV
csvMerged = "Wednesday100124.csv"

In [7]:
# This cell can take upto 20 min to load
def convert_pcapng_to_csv(pcapng_file):
    # Full path to the pcapng file
    pcapng_path = os.path.join(pcapng_dir, pcapng_file)

    if not os.path.exists(pcapng_path):
        print(f"Error: File {pcapng_path} does not exist.")
        return

    csv_path = os.path.join(csv_dir, pcapng_file + '.csv')

    commandPcapngToCsv = [
        "C:\\Program Files\\Wireshark\\tshark.exe",
        "-r",
        pcapng_path,
        "-T",
        "fields",
        "-E",
        "header=y",
        "-E",
        "separator=,",
        "-E",
        "quote=d",
        "-E",
        "occurrence=f",
        "-e",
        "frame.time",
        "-e",
        "wlan.sa",
        "-e",
        "wlan.da",
        "-e",
        "_ws.col.Protocol",
        "-e",
        "wlan.ssid",
        "-e",
        "wlan.bssid",
        "-e",
        "wlan.seq",
        "-e",
        "wlan.fc.type_subtype"
    ]

    try:
        with open(csv_path, "w") as f:
            subprocess.run(commandPcapngToCsv, stdout=f, check=True)
    except Exception as e:
        print(f"Error processing {pcapng_file}: {e}")


# Example usage with concurrent.futures
pcapng_files = [f for f in os.listdir(pcapng_dir)]

with mp.ThreadPoolExecutor(max_workers=16) as executor:
    try:
        executor.map(convert_pcapng_to_csv, pcapng_files)
    except Exception as e:
        print(f"Error processing: {e}")


In [8]:
# Get a list of all csv files in the directory
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]

# # Read each csv file and append it to the list of dataframes
dfs = [pd.read_csv(os.path.join(csv_dir, csv_file), low_memory=False) for csv_file in csv_files]

# Concatenate all dataframes into one
df = pd.concat(dfs, ignore_index=True)

# Save the combined dataframe to a new csv file
df.to_csv(os.path.join(csv_dir, csvMerged), index=False)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 199682: invalid continuation byte

In [5]:
df['wlan.sa'].nunique()

326030